In [12]:
import os
import numpy as np
import pandas as pd
from zipfile import ZipFile
import wget
import requests
from bs4 import BeautifulSoup as soup

import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300
plt.rcParams['figure.figsize'] = (15, 8)
plt.style.use('seaborn-white')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
lista_empresas = pd.read_csv('match_perfeito.csv', sep=';', encoding='latin1')
lista_empresas = lista_empresas.iloc[:, [1, 3, 4]]
lista_empresas.columns = ['Ticker', 'Empresa', 'CNPJ']
lista_empresas.set_index('Ticker', inplace=True)

In [ ]:
url_lista_empresas = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv'
wget.download(url_lista_empresas)

In [16]:
urls = [('BPA', 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/BPA/DADOS/'),
        ('BPP', 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/BPP/DADOS'),
        ('DRE', 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DRE/DADOS/'),
        ('DFC_MD', 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DFC_MD/DADOS/'),
        ('DFC_MI', 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DFC_MI/DADOS/'),
        ('ITR', 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')]

In [17]:
for url in urls:
    if not 'ITR' in os.listdir():
        os.makedirs(url[0])
    
    print(url[1])
    r = requests.get(url[1])
    p = soup(r.content, 'html.parser')
    tabela = p.find('table', {'id': 'indexlist'})
    linhas = tabela.find_all('tr')[3:]
    
    lista = []
    for linha in linhas:
        try:
            arq = linha.find_all('td')[1].a['href']
            lista.append(arq)
        except:
            pass
        
    for i in lista:
        if not i in os.listdir(url[0]):
            wget.download(url[1] + i, url[1])

http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/BPA/DADOS/
http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/BPP/DADOS
http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DRE/DADOS/
http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DFC_MD/DADOS/
http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DFC_MI/DADOS/
http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/


In [18]:
RESULTADO = {}

In [19]:
def get_ITR_data(cnpj, fillings=['BPA', 'BPP']):
    global RESULTADO
    lista_arquivos = {}
    
    def open_zip_file(arq_zip, nome_csv):
        with ZipFile(arq_zip, 'r') as arquivo:
            if nome_csv in arquivo.namelist(): 
                pass
            
            elif nome_csv not in arquivo.namelist():
                nome_csv = nome_csv.lower()
        
            with arquivo.open(nome_csv) as f:
                tabela = pd.read_csv(f, sep=';', encoding='latin1')
        
        return tabela
    
    def find_data(tabela):
        cnpjs = tabela['CNPJ_CIA'].unique()
        datas = tabela['DT_FIM_EXERC'].unique()
        
        if not cnpj in RESULTADO:
            RESULTADO[cnpj] = {}
        
        if not filling in RESULTADO[cnpj]:
            RESULTADO[cnpj][filling] = {}

        for data in datas:
            t = tabela[(tabela['CNPJ_CIA']==cnpj) & (tabela['DT_FIM_EXERC']==data)].copy()
            
            if not data in RESULTADO[cnpj][filling]:
                RESULTADO[cnpj][filling][data] = {}
            
            for i in range(t.shape[0]):
                RESULTADO[cnpj][filling][data][t.iloc[i]['DS_CONTA']] = t.iloc[i]['VL_CONTA']
    
    for filling in fillings:
        lista_arquivos[filling] = [i for i in os.listdir(filling) if i[-4:] == '.zip']
        lista_arquivos['ITR'] = [i for i in os.listdir('ITR') if i[-4:] == '.zip']
        
        for arq_zip in lista_arquivos[filling]:
            nome_csv = '%s_CIA_ABERTA_con_%s.csv' % (filling, arq_zip[-8:-4])    
            tabela = open_zip_file(filling + '/' + arq_zip, nome_csv)
            find_data(tabela)
        
        for arq_zip in lista_arquivos['ITR']:
            nome_csv = 'ITR_CIA_ABERTA_%s_con_%s.csv' % (filling, arq_zip[-8:-4])
            tabela = open_zip_file('ITR/' + arq_zip, nome_csv)
            find_data(tabela)

In [20]:
def desacumular(df):
    x = df.copy()
    x_ = df.copy()
    for c in x.columns:
        for i in range(x.shape[0]):
            month = x.iloc[i].name.month
            year = x.iloc[i].name.year

            if month > 3:
                try:
                    x_.loc['%d-%d' % (year, month), c] = x.loc['%d-%d' % (year, month), c].values - x.loc['%d-%d' % (year, month-3), c].values
                except:
                    pass
                
    return x_

In [23]:
class Empresa:
    
    def __init__(self, ticker, cnpj=''):
        self.ticker = ticker
        self.name = lista_empresas.loc[ticker]['Empresa']
        
        if cnpj == '': self.cnpj = lista_empresas.loc[ticker]['CNPJ']
        else: self.cnpj = cnpj
        
    def get_balance_sheet(self):
        get_ITR_data(self.cnpj)
        BPA = pd.DataFrame(RESULTADO[self.cnpj]['BPA']).dropna(axis=1, how='all')
        BPP = pd.DataFrame(RESULTADO[self.cnpj]['BPP']).dropna(axis=1, how='all')
        
        self.BP = pd.concat([BPA, BPP], axis=0).T.sort_index().T
        return self.BP
    
    def get_income_statement(self):
        get_ITR_data(self.cnpj, fillings=['DRE'])
        t_ = pd.DataFrame(RESULTADO[self.cnpj]['DRE']).dropna(axis=1, how='all').T.sort_index().reset_index()
        t_['index'] = pd.to_datetime(t_['index'])
        t_ = t_.set_index('index')
            
        self.DRE = desacumular(t_).T.copy()
        return self.DRE
        
    def get_dfc_md(self):
        get_ITR_data(self.cnpj, fillings=['DFC_MD'])
        t_ = pd.DataFrame(RESULTADO[self.cnpj]['DFC_MD']).dropna(axis=1, how='all').T.sort_index().reset_index()
        t_['index'] = pd.to_datetime(t_['index'])
        t_ = t_.set_index('index')
            
        self.DFC_D = desacumular(t_).T.copy()
        return self.DFC_D
        
    def get_dfc_mi(self):
        get_ITR_data(self.cnpj, fillings=['DFC_MI'])
        t_ = pd.DataFrame(RESULTADO[self.cnpj]['DFC_MI']).dropna(axis=1, how='all').T.sort_index().reset_index()
        t_['index'] = pd.to_datetime(t_['index'])
        t_ = t_.set_index('index')
            
        self.DFC_I = desacumular(t_).T.copy()
        return self.DFC_I

In [34]:
t = Empresa('ABEV3')

In [36]:
t.get_income_statement()

index,2012-06-30,2012-09-30,2012-12-31,2013-03-31,2013-06-30,2013-09-30,2013-12-31,2014-03-31,2014-06-30,2014-09-30,...,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31
Receita de Venda de Bens e/ou Serviços,14061117.0,8036022.0,10133888.0,7832000.0,7570220.0,8544259.0,11132628.0,9045072.0,8177433.0,8624396.0,...,11362303.0,15027215.0,11640219.0,11509545.0,11063743.0,16017829.0,12640126.0,12145133.0,11957652.0,15856798.0
Custo dos Bens e/ou Serviços Vendidos,-4703366.0,-2667728.0,-3088692.0,-2697779.0,-2667024.0,-2884137.0,-3323603.0,-3008314.0,-3040666.0,-2955760.0,...,-4482122.0,-4988326.0,-4455934.0,-4382306.0,-4365818.0,-6045365.0,-5107701.0,-4961301.0,-5229732.0,-6379425.0
Resultado Bruto,9357751.0,5368294.0,7045196.0,5134221.0,4903196.0,5660122.0,7809025.0,6036758.0,5136767.0,5668636.0,...,6880181.0,10038889.0,7184285.0,7127239.0,6697925.0,9972464.0,7532425.0,7183832.0,6727920.0,9477373.0
Despesas/Receitas Operacionais,-3946153.0,-2099358.0,-2095410.0,-2021782.0,-2201410.0,-1973603.0,-1867578.0,-2519280.0,-2367188.0,-2175213.0,...,-3270946.0,-3769479.0,-3382951.0,-3441561.0,-3292941.0,-3712557.0,-3478828.0,-3636384.0,-3595705.0,-4158061.0
Despesas com Vendas,-3552955.0,-1821806.0,-1976159.0,-1993527.0,-2092677.0,-2009291.0,-1964421.0,-2315433.0,-2347222.0,-2159875.0,...,-2971041.0,-3168599.0,-3061504.0,-3142127.0,-2859329.0,-3265551.0,-3028003.0,-3085118.0,-3078160.0,-3456255.0
Despesas Gerais e Administrativas,-674200.0,-519477.0,-409872.0,-353084.0,-403748.0,-355599.0,-635909.0,-443428.0,-354940.0,-394205.0,...,-528467.0,-1018890.0,-571192.0,-581644.0,-615809.0,-594820.0,-661522.0,-707400.0,-632466.0,-678579.0
Perdas pela Não Recuperabilidade de Ativos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Outras Receitas Operacionais,347495.0,211751.0,410531.0,328786.0,303548.0,508249.0,852810.0,292404.0,422247.0,404289.0,...,289770.0,476209.0,307434.0,463338.0,266181.0,390716.0,285051.0,273442.0,154355.0,393916.0
Outras Despesas Operacionais,-66552.0,30142.0,-120300.0,-5645.0,-8630.0,-120854.0,-125800.0,-60693.0,-89573.0,-28465.0,...,-56673.0,-53318.0,-58306.0,-183683.0,-80712.0,-244042.0,-72218.0,-116577.0,-31186.0,-405948.0
Resultado de Equivalência Patrimonial,59.0,32.0,390.0,1688.0,97.0,3892.0,5742.0,7870.0,2300.0,3043.0,...,-4535.0,-4881.0,617.0,2555.0,-3272.0,1140.0,-2136.0,-731.0,-8248.0,-11195.0
